In [5]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_C", "B_P", "I_C", "I_P"]
ner_args = NERArgs()
ner_args.max_seq_length = 480
test_ner = NERModel("camembert", 'pitiwat/argument_wangchanberta2', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

In [6]:
def tag_html_format(pred_list):
    LIST_TAGS = ['claim', 'premise', 'o']
    REPRESEN_TAGS = ['c', 'p', 'O']
    text_convert = ''
    prev_tag = ''
    trigger_tag = False 


    for word, label in pred_list:
        tags = label.split('-')
        next_tag = tags[0] if len(tags) == 1 else tags[1]
        
        if prev_tag != next_tag:
            if prev_tag:
                text_convert += '</' + html_tag + '>'

            html_tag = LIST_TAGS[REPRESEN_TAGS.index(next_tag)]
            prev_tag = next_tag
            trigger_tag = not(trigger_tag)

            if trigger_tag:
                text_convert += '<' + html_tag + '>'
            else:
                text_convert += '<' + html_tag + '>'

        text_convert += word
    text_convert += '</'+ LIST_TAGS[REPRESEN_TAGS.index(prev_tag)]+'>'
    text_convert = text_convert.replace('<o>', '').replace('</o>', '')
            
    return text_convert

In [7]:
def predict_text(text):
    predict_text = []
    temp = []
    text_token = word_tokenize(text)
    text_token = [word if word.strip() != '' else '_' for word in text_token]
    predictions, raw_outputs = test_ner.predict([text_token], split_on_space=False)
    # convert [{}] to [()] and convert B_C to B-c
    for pred_dict in predictions[0]:
        for word, label in pred_dict.items():
             label = label.capitalize().replace('_', '-')
             predict_text.append((word, label))
    predict_text = [(word, label) if word != '_' else (' ', label) for word, label in predict_text]
    return predict_text, tag_html_format(predict_text)

In [8]:
from pythainlp.tokenize import word_tokenize

text = '''ศาสนาพุทธกับวิทยาศาสตร์มีส่วนที่เกี่ยวกันอยู่ ทั้งในแง่ของศาสตร์ และกระบวนการ
ศาสนากับวิทยาศาสตร์ ไม่ได้ขัดแย้งกัน แต่เพียงอธิบายสิ่งต่าง ๆ ออกมาไม่เหมือนกันโดยผ่านมนุษย์
มนุษย์ใช้ศาสนาอธิบายบางสิ่งที่วิทยาศาสตร์ได้อธิบายไว้ว่าเป็นอย่างนั้นอย่างนี้
และมนุษย์ก็ใช้วิทยาศาสตร์อธิบายสิ่งที่ศาสนาได้อธิบายไว้ว่าเป็นอย่างนั้นอย่างนี้
ซึ่งบางครั้งก็สนับสนุนกัน และบางครั้งก็อาจจะขัดแย้งกัน (ขัดแย้งกันในที่นี้ไม่ได้หมายถึงศาสนากับวิทยาศาสตร์ แต่ขัดแย้งกันในมุมของมนุษย์)
จนบางครั้งก็เกิดเป็นลัทธิประหลาดที่นำเอาศาสนากับวิทยาศาสตร์มาบังหน้า
ที่มันเป็นอย่างนี้ก็เพราะว่าไม่มีความเข้าใจอย่างถ่องแท้เกี่ยวกับศาสนาและวิทยาศาสตร์ ซึ่งมนุษย์ก็ต้องหาทางต่าง ๆ ต่อไป เพื่อการพัฒนา ไม่ว่าทางใด'''
# text ="ฉันชอบหมาเพราะมันน่ารัก"

pred_text, pred_tag = predict_text(text)
print(pred_text)
print("----")
print(pred_tag)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[('ศาสนาพุทธ', 'B-c'), ('กับ', 'I-c'), ('วิทยาศาสตร์', 'I-c'), ('มีส่วน', 'I-c'), ('ที่', 'I-c'), ('เกี่ยว', 'I-c'), ('กัน', 'I-c'), ('อยู่', 'I-c'), (' ', 'O'), ('ทั้ง', 'B-p'), ('ในแง่', 'O'), ('ของ', 'I-p'), ('ศาสตร์', 'I-p'), (' ', 'O'), ('และ', 'O'), ('กระบวนการ', 'I-p'), (' ', 'O'), ('ศาสนา', 'B-p'), ('กับ', 'I-c'), ('วิทยาศาสตร์', 'I-c'), (' ', 'I-p'), ('ไม่', 'I-c'), ('ได้', 'I-p'), ('ขัดแย้ง', 'I-p'), ('กัน', 'I-p'), (' ', 'I-p'), ('แต่เพียง', 'I-p'), ('อธิบาย', 'I-p'), ('สิ่ง', 'I-p'), ('ต่าง ๆ', 'I-p'), (' ', 'I-p'), ('ออกมา', 'I-p'), ('ไม่', 'I-p'), ('เหมือนกัน', 'I-p'), ('โดย', 'I-p'), ('ผ่าน', 'I-p'), ('มนุษย์', 'I-p'), (' ', 'I-p'), ('มนุษย์', 'I-p'), ('ใช้', 'I-p'), ('ศาสนา', 'I-p'), ('อธิบาย', 'I-p'), ('บางสิ่ง', 'I-p'), ('ที่', 'I-p'), ('วิทยาศาสตร์', 'I-p'), ('ได้', 'I-p'), ('อธิบาย', 'I-p'), ('ไว้', 'I-p'), ('ว่า', 'I-p'), ('เป็น', 'I-p'), ('อย่างนั้น', 'I-p'), ('อย่างนี้', 'I-p'), (' ', 'I-p'), ('และ', 'I-p'), ('มนุษย์', 'I-p'), ('ก็', 'I-p'), ('ใช้', 'I-p'), ('วิท

In [11]:
print(' '.join(word_tokenize(text)))

ศาสนาพุทธ กับ วิทยาศาสตร์ มีส่วน ที่ เกี่ยว กัน อยู่   ทั้ง ในแง่ ของ ศาสตร์   และ กระบวนการ 
 ศาสนา กับ วิทยาศาสตร์   ไม่ ได้ ขัดแย้ง กัน   แต่เพียง อธิบาย สิ่ง ต่าง ๆ   ออกมา ไม่ เหมือนกัน โดย ผ่าน มนุษย์ 
 มนุษย์ ใช้ ศาสนา อธิบาย บางสิ่ง ที่ วิทยาศาสตร์ ได้ อธิบาย ไว้ ว่า เป็น อย่างนั้น อย่างนี้ 
 และ มนุษย์ ก็ ใช้ วิทยาศาสตร์ อธิบาย สิ่ง ที่ ศาสนา ได้ อธิบาย ไว้ ว่า เป็น อย่างนั้น อย่างนี้ 
 ซึ่ง บางครั้ง ก็ สนับสนุน กัน   และ บางครั้ง ก็ อาจจะ ขัดแย้ง กัน   ( ขัดแย้ง กัน ในที่นี้ ไม่ ได้ หมายถึง ศาสนา กับ วิทยาศาสตร์   แต่ ขัดแย้ง กัน ใน มุม ของ มนุษย์ ) 
 จน บางครั้ง ก็ เกิด เป็น ลัทธิ ประหลาด ที่ นำ เอา ศาสนา กับ วิทยาศาสตร์ มา บังหน้า 
 ที่ มัน เป็น อย่างนี้ ก็เพราะว่า ไม่ มี ความเข้าใจ อย่าง ถ่องแท้ เกี่ยวกับ ศาสนา และ วิทยาศาสตร์   ซึ่ง มนุษย์ ก็ ต้อง หาทาง ต่าง ๆ   ต่อไป   เพื่อ การพัฒนา   ไม่ ว่า ทาง ใด
